# Projeto Telecom X — Análise de Evasão de Clientes (Churn)

**Autor original:** Paulo Henrique Santana Motta  
**Melhorias aplicadas:** Refatoração do fluxo ETL, funções reutilizáveis, feature engineering, pipeline de ML (Logistic Regression), avaliação robusta, visualizações aprimoradas e seção de conclusões.

Este notebook foi gerado automaticamente para melhorar a estrutura e a reprodutibilidade do projeto.


## Sumário

1. Setup e imports
2. Carregamento dos dados
3. Funções utilitárias (ETL & Preprocessing)
4. Análise Exploratória (EDA)
5. Feature Engineering
6. Modelagem Preditiva (Pipeline)
7. Avaliação e Métricas
8. Insights e Conclusões

---


In [ ]:
# 1) Setup e imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, roc_curve, auc)

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

# Fonte dos dados (originalmente usado no projeto)
url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/main/TelecomX_Data.json"
print('Setup pronto. URL dos dados:', url)


In [ ]:
# 2) Carregamento dos dados

df = pd.read_json(url)
print('Dimensão do dataset:', df.shape)
df.head()


In [ ]:
# 3) Funções utilitárias: limpeza, tratamento e feature engineering

def quick_report(df):
    print('--- RESUMO ---')
    print('Linhas, colunas:', df.shape)
    print('\nTipos de dados:')
    print(df.dtypes)
    print('\nValores ausentes por coluna:')
    print(df.isnull().sum().sort_values(ascending=False).head(20))


def preprocess_basic(df):
    df = df.copy()
    # Remover duplicatas
    df = df.drop_duplicates()

    # Converter colunas numéricas mal interpretadas
    # (aplica-se conforme existam strings em colunas numéricas)
    for col in df.select_dtypes(include=['object']).columns:
        # tentar converter para num quando for completamente numérico
        try:
            coerced = pd.to_numeric(df[col], errors='coerce')
            non_null_ratio = coerced.notnull().mean()
            if non_null_ratio > 0.95 and coerced.nunique() > 5:
                df[col] = coerced
        except Exception:
            pass

    return df


def feature_engineer(df):
    df = df.copy()
    # Exemplo de features simples
    if 'MonthlyCharges' in df.columns and 'TotalCharges' in df.columns:
        df['avg_months'] = df['TotalCharges'] / (df['MonthlyCharges'].replace(0, np.nan))
        df['avg_months'] = df['avg_months'].fillna(0).replace([np.inf, -np.inf], 0)
    # Binarizar churn se ainda não estiver 0/1
    if 'Churn' in df.columns:
        df['Churn_flag'] = df['Churn'].map({'Yes':1, 'No':0}) if df['Churn'].dtype == object else df['Churn']
    return df

print('Funções utilitárias definidas.')


In [ ]:
# 4) Análise Exploratória Rápida (EDA)

quick_report(df)

# Distribuição da variável alvo (Churn)
if 'Churn' in df.columns:
    display(df['Churn'].value_counts(dropna=False))

# Algumas visualizações rápidas
plt.figure(figsize=(6,4))
if 'Churn' in df.columns:
    df['Churn'].value_counts(normalize=True).plot(kind='bar')
    plt.title('Distribuição de Churn')
    plt.xlabel('Churn')
    plt.ylabel('Proporção')
    plt.show()

# Correlação entre numéricas
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    print('\nTop 10 correlações com Churn_flag (se existir):')
    if 'Churn_flag' in df.columns:
        print(corr['Churn_flag'].abs().sort_values(ascending=False).head(10))




In [ ]:
# 5) Aplicar pré-processamento e feature engineering

df_clean = preprocess_basic(df)
df_fe = feature_engineer(df_clean)
print('Dimensão após FE:', df_fe.shape)
df_fe.head()


In [ ]:
# 6) Modelagem preditiva — Pipeline simples (Logistic Regression)

# Preparar X e y
if 'Churn_flag' not in df_fe.columns:
    raise ValueError('Coluna Churn_flag ausente — verifique o dataset e a etapa de feature engineering')

y = df_fe['Churn_flag']
X = df_fe.drop(columns=['Churn', 'Churn_flag']) if 'Churn' in df_fe.columns else df_fe.drop(columns=['Churn_flag'])

# Selecionar tipos
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print('Numérico:', len(numeric_features), 'Categórico:', len(cat_features))

# Simple pipeline
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
cat_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', cat_transformer, cat_features)
])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(max_iter=1000))])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

# Treinar
clf.fit(X_train, y_train)
print('Treinamento concluído.')

# Prever
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]

print('Acurácia (teste):', accuracy_score(y_test, y_pred))
print('\nRelatório de classificação:\n', classification_report(y_test, y_pred))
print('\nROC-AUC:', roc_auc_score(y_test, y_proba))


In [ ]:
# 7) Avaliação — Matriz de confusão e ROC

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
plt.imshow(cm, interpolation='nearest')
plt.title('Matriz de Confusão')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center')
plt.colorbar()
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1], linestyle='--')
plt.title('ROC Curve (AUC = {:.3f})'.format(roc_auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# Top features (apenas para coeficientes do modelo linear)
try:
    # extrair feature names do preprocessor
    ohe_cols = []
    if len(cat_features) > 0:
        ohe = clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
        cat_names = list(ohe.get_feature_names_out(cat_features))
    else:
        cat_names = []
    feat_names = numeric_features + cat_names
    coefs = clf.named_steps['classifier'].coef_[0]
    feat_imp = pd.DataFrame({'feature': feat_names, 'coef': coefs})
    feat_imp['abs_coef'] = feat_imp['coef'].abs()
    feat_imp = feat_imp.sort_values('abs_coef', ascending=False).head(15)
    print('\nTop features por importância (coef):')
    display(feat_imp)
except Exception as e:
    print('Não foi possível extrair importâncias automaticamente:', e)


## 8) Insights e Conclusões

- **Resumo da modelagem:** Implementamos um pipeline simples com pré-processamento (escalonamento + one-hot) e um classificador linear (Logistic Regression). Esta abordagem é rápida, interpretável e fornece baseline robusto.
- **Próximos passos recomendados:**
  - Balanceamento (SMOTE / undersampling) caso haja desbalanceamento forte.
  - Testar modelos mais complexos (RandomForest, XGBoost) e comparar via CV.
  - Ajustar features (feature selection, encoding target/ordinal quando aplicável).
  - Criar dashboard interativo (Plotly / Streamlit) para stakeholders.
